# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Prompt Engineering for Instruction-Tuned LLM: Text Summarization & Information Retrieval </center>
***


In today's fast-paced world, we're inundated with an overwhelming amount of text, leaving little time to read everything we desire. A fascinating application of large language models is their use in text summarization and information retrieval. 
Multiple teams are incorporating this feature into various software applications, including the chatGPT web interface. You can utilize LLM to summarize articles, enabling you  to consume the content of numerous articles more efficiently. If you're interested in a more programmatic approach, this article provides guidance on how to achieve that using prompt engineering.

#### <a id="top"></a>

# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Setting Working Environment & Getting Started </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. Summarize with Specific Purpose </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3.  Information Retrieval </a></li> 
    <li><a href="#4" target="_self" rel=" noreferrer nofollow">4. Summarize Multiple Texts </a></li>
    
</ul>
</div>

***


<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Setting Working Environment & Getting Started </b></div>


We will use the OpenAI Python library to access the OpenAI API. You can this Python library using pip like this:
    

In [1]:
!pip install openai

Next, we will import OpenAI and then set the OpenAI API key which is a secret key. You can get one of these API keys from the OpenAI website. It is better to set this as an environment variable to keep it safe if you share your code. We will use OpenAI's chatGPT GPT 3.5 Turbo model, and the chat completions endpoint.

In [2]:
import os
import utils
import panel as pn
from openai import OpenAI
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI

# ==============================
# 🔑 Load environment variables
# ==============================
env_values = dotenv_values(".env")

openai_api_key = env_values.get("OPENAI_API_KEY")
openai_api_base = env_values.get("OPENAI_API_BASE")
openai_api_name = env_values.get("OPENAI_API_NAME")

# Initialize Clients
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base
)

# Mock moderation for OpenRouter compatibility
if openai_api_base and "openrouter.ai" in openai_api_base:
    class MockResult(dict):
        def __getattr__(self, name): return self.get(name)
    class MockModeration(dict):
        def __init__(self):
            res = MockResult(flagged=False, categories={}, category_scores={})
            super().__init__(results=[res])
        @property
        def results(self): return self["results"]
    client.moderations.create = lambda *args, **kwargs: MockModeration()

chat = ChatOpenAI(
    model_name=openai_api_name, 
    openai_api_key=openai_api_key, 
    openai_api_base=openai_api_base,
    temperature=0.7
)
pn.extension()

# ==============================
# 🔧 Helper Functions
# ==============================
def get_completion(prompt, model=openai_api_name, temperature=0, max_tokens=500):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model=openai_api_name, temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6388\312141790.py:41: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


Finally, we will define a helper function to make it easier to use prompts and look at generated outputs. So that's this function, getCompletion, that just takes in a prompt and will return the completion for that prompt.

In [3]:
# Function definitions moved to setup cell above

Let's start with an example of the task of summarizing a movie  review. If you're building a movie streaming website, and there's just a large volume of reviews, having a tool to summarize the lengthy reviews could give you a way to very quickly glance over more reviews to get a better sense of what all your customers are thinking.

In [4]:
movie_review = """
Watched this movie over the weekend, and it turned out to be a delightful cinematic experience. The storyline was engaging, weaving together various plot elements seamlessly. The characters were well-developed, and their interactions felt genuine. The film had a perfect blend of drama, humor, and suspense, keeping me hooked from start to finish.

The visuals were stunning, with impressive cinematography that added depth to the narrative. The special effects were top-notch, enhancing the overall viewing experience. The soundtrack was also noteworthy, complementing the on-screen action and evoking the right emotions at the right moments.

Despite these strengths, there were a few drawbacks. Some scenes felt a bit rushed, leaving me wanting more in terms of character development. Additionally, the ending seemed a tad predictable, lacking the surprise factor that could have elevated the film to greater heights.

On the positive side, the movie was a bit of a hidden gem, exceeding my expectations. The pacing was well-managed, and the runtime felt just right. It left me with a sense of satisfaction, much like finding a surprise extra feature on a DVD.

In conclusion, this film is worth a watch for its compelling story, impressive visuals, and engaging characters. While it may not be a groundbreaking masterpiece, it certainly provides an enjoyable and entertaining escape into its cinematic world.

"""

This is a very long review so let's write a prompt to summarize this review in less than 30 words only.

In [5]:
prompt = f"""
Your task is to generate a short summary of a movie \
review from a streaming service. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{movie_review}```
"""

response = get_completion(prompt)
print(response)

The film offers an engaging story, stunning visuals, and well-developed characters, though some scenes feel rushed and the ending is predictable. Overall, it's an enjoyable cinematic experience.


It's a good summary although it seems a little bit longer than 30 words but it is still acceptable. You can also change  the character count or the number of sentences to affect the length of this summary. Sometimes when creating a summary, if you have a very specific purpose in mind for the summary. Let's see how we can do this.

<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Summarize with Specific Purpose  </b></div>



If you want to give feedback to a specific  department or to get feedback on a specific movie criteria, you can modify the prompt to reflect that, so that they can generate a summary that is more applicable to one particular group in your business. 
So, for example, I add to get feedback on the movie's ending sequence to enhance the movie recommendation system. I will change the prompt to start focusing on any aspects that mention the movie's ending sequence.

In [6]:
prompt = f"""
Your task is to extract relevant information from \ 
a movie review from a streaming service to give to the recommendation engine \

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on  \
how the watcher felt about the movie sequence ending.

Review: ```{movie_review}```
"""

response = get_completion(prompt)
print(response)

The ending felt predictable and lacked surprise, but overall, the movie left the viewer satisfied and exceeded expectations.


Again you get a summary and it now focuses on the fact that the ending sequence was rushed and the predictable and lacking surpirse.
Let's take another example. Let's say we want to get feedback on the character's development and how far the watcher engaged with them. I'm going to tell it to focus on any aspects that are relevant to the characters

In [7]:
prompt = f"""
Your task is to extract relevant information from \ 
a movie review from a streaming service to give the recommendation engine \

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing how the watcher\
feels about the movie characters. 

Review: ```{movie_review}```
"""

response = get_completion(prompt)
print(response)

The characters are well-developed and their genuine interactions enhance the film, making it an engaging and satisfying experience despite minor drawbacks.


You can try to summarize based on other aspects of the movie and see how it works.

<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Information Retrieval </b></div>


In the previous summaries, even though it generated the information relevant to the ending sequence we can see that it had some other information too, which you could decide may or may not be helpful. So, depending on how you want to summarize it, you can also ask it to extract information rather than summarize it.
So here's a prompt that says your  task is to extract relevant information to about the movie's ending sequence.

In [8]:
prompt = f"""
Your task is to extract relevant information from \ 
a movie review from a streaming service.

From the review below, delimited by triple quotes \
extract the information relevant to ending sequence \ 
Limit to 30 words. 

Review: ```{movie_review}```
"""

response = get_completion(prompt)
print(response)

The ending seemed predictable, lacking the surprise factor that could have elevated the film, but it left a sense of satisfaction, like finding a surprise extra feature.


This can be used as an information retrieval model which is a complex NLP task. Now you can do it in a very simple way using only prompt engineering.

<a id="4"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 4. Summarize Multiple Texts  </b></div>


Lastly, let's take a concrete example of how to use this in a workflow to help summarize multiple reviews to make them easier to read. 
So, here are four movie reviews and we will use the prompt to summarize each of them.

In [9]:
review_1 = movie_review 

# review for moview 2
review_2 = """
I was completely immersed in this cinematic gem! 
The storytelling is so engaging, and the characters feel like old friends. 
The mix of drama, humor, and suspense had me hooked throughout. 
Visually, it's a feast for the eyes with stunning effects, and the 
soundtrack added so much emotion. Sure, the ending was a bit expected, 
but the overall experience was so enjoyable that I highly recommend giving 
it a watch
"""

# review for movie 3
review_3 = """
What a journey! I felt like I was right there with 
the characters in this film. The storytelling is fantastic, and the 
characters are so well-developed that I genuinely cared about their fates. 
The blend of drama, humor, and suspense kept me entertained, 
and the visuals were breathtaking. Despite a few rushed scenes, 
I found myself thoroughly enjoying this movie. 
It's definitely worth your time.
"""

# review for movie 4
review_4 = """
I had a blast watching this film! The storytelling drew me in, 
and the characters were so relatable. The perfect mix of drama, humor, 
and suspense made it a rollercoaster of emotions. Visually, it's stunning, 
and the soundtrack added an extra layer of excitement. 
While the ending wasn't a huge surprise, 
the overall experience left me thoroughly entertained. 
If you're looking for a fun and engaging movie night, 
this one's a solid choice.
"""

reviews = [review_1, review_2, review_3, review_4]

This is a lot of text. But what if you want to know what these reviewers wrote without having to stop and read all this in detail? So, I'm going to set review one to be just the movie review that we had up there. And I'm going to put all of these reviews into a list. And now, if I implement or loop over the reviews, so, here's my prompt. And here I've asked it to summarize it in at most 20 words. Then let's have it get the response and print it out. And let's run that.

In [10]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to extract relevant information from \ 
    a movie review from a streaming service.
    
    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 Delightful film with engaging story, stunning visuals, and well-developed characters, despite some rushed scenes and a predictable ending. 



1 Engaging storytelling, stunning visuals, and a captivating mix of drama and humor make it highly recommended. 



2 Engaging storytelling and well-developed characters make this movie a worthwhile watch despite some rushed scenes. 



3 Engaging film with relatable characters, stunning visuals, and a mix of drama, humor, and suspense. Highly recommended! 



If you have any applications with long pieces of text, you can use prompts like these to summarize them and get information out of them to help people quickly understand the text and perhaps optionally dig in more if they wish. 
In the next article, we'll look at another capability of large language models, which is to make inferences using text. For example, what if you had, again, movie reviews and you wanted to very quickly get a sense of which movie reviews have a positive or negative sentiment or do sentiment analysis of the movie reviews?